## Исследование надёжности заёмщиков

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели **кредитного скоринга** — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

### Шаг 1. Откройте файл с данными и изучите общую информацию. 

In [1]:
import pandas as pd
df = pd.read_csv('/datasets/data.csv')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


In [2]:
df.head(15)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


### Вывод

Рассмотрим полученную информацию подробнее.
Всего в таблице 12 столбцов.
Подробно разберём, какие в df столбцы и какую информацию они содержат:
    
    children — количество детей в семье
    days_employed — общий трудовой стаж в днях
    dob_years — возраст клиента в годах
    education — уровень образования клиента
    education_id — идентификатор уровня образования
    family_status — семейное положение
    family_status_id — идентификатор семейного положения
    gender — пол клиента
    income_type — тип занятости
    debt — имел ли задолженность по возврату кредитов
    total_income — ежемесячный доход
    purpose — цель получения кредита
    
Названия столбцов написаны корректно. Количество значений в столбцах различается. Это говорит о том, что в данных есть пропущенные значения. 

### Шаг 2. Предобработка данных

### Обработка пропусков

Проверим данные на наличие пропусков

In [3]:
df.isnull().sum()

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

Удаляем пропуски в столбцах, т.к в процентном соотношении оно составляют 1%. И убеждаемся что их больше не осталось.

In [4]:
df['total_income'] = df['total_income'].fillna(df.groupby('income_type')['total_income'].transform('median'))

In [5]:
df['days_employed'] = df['days_employed'].abs()
df['days_employed'] = df['days_employed'].fillna(df.groupby('income_type')['days_employed'].transform('median'))

In [6]:
df.isnull().sum()

children            0
days_employed       0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
dtype: int64

### Вывод

Пробуски в столбцах "общий трудовой стаж в днях" и "ежемесячный доход" составляют 1% от всех значений датафрейма, в котором 21525 строк. Удаление таких строк не повлияет на результат.

### Замена типа данных

Посмотрим какие данные в столбце 'children'.

In [7]:
df['children'].value_counts()

 0     14149
 1      4818
 2      2055
 3       330
 20       76
-1        47
 4        41
 5         9
Name: children, dtype: int64

Отрицательное значение свидетельсвует об ошибочном значении. Переведем значения в ячеке количества детей с -1 на 1. В процентном соотношении ячеек с отрицательным значением очень мало, объединив их в одну группу с одним ребенком, на результат анализа не повлияем.

In [8]:
df['children'] = df['children'].abs()

Проанализировав данные о клиентах банка с количеством детей 20, приходим к выводу, что такое количество детей указано ошибочно. Цифры 2 и 0 на раскалдке клавиатуры находятся рядом и клиенты скорей всего нажали одновременнго две клавиши, поэтому объеденим их в группу по 2 ребенка в семье.

In [9]:
df['children'] = df['children'].replace(20, 2)
df['children'].value_counts()

0    14149
1     4865
2     2131
3      330
4       41
5        9
Name: children, dtype: int64

In [10]:
df['total_income'] = df['total_income'].astype('int')
df['total_income'].mean()

165224.81602787456

### Вывод

Некотрые ошибочные значения имеют человеческий фактор происхджденияю. Ежемесячный доход для удобства чтения преобразуем в целочисленные значения.

### Обработка дубликатов

Проверим наличие дубликатов в столбце 'education'.

In [11]:
df['education'].value_counts()

среднее                13750
высшее                  4718
СРЕДНЕЕ                  772
Среднее                  711
неоконченное высшее      668
ВЫСШЕЕ                   274
Высшее                   268
начальное                250
Неоконченное высшее       47
НЕОКОНЧЕННОЕ ВЫСШЕЕ       29
НАЧАЛЬНОЕ                 17
Начальное                 15
ученая степень             4
Ученая степень             1
УЧЕНАЯ СТЕПЕНЬ             1
Name: education, dtype: int64

Проверим наличие дубликатов в столбце 'family_status'.

In [12]:
df['family_status'].value_counts()

женат / замужем          12380
гражданский брак          4177
Не женат / не замужем     2813
в разводе                 1195
вдовец / вдова             960
Name: family_status, dtype: int64

Проверим наличие дубликатов в столбце 'income_type'.

In [13]:
df['income_type'].value_counts()

сотрудник          11119
компаньон           5085
пенсионер           3856
госслужащий         1459
безработный            2
предприниматель        2
студент                1
в декрете              1
Name: income_type, dtype: int64

Проверим наличие дубликатов в столбце 'purpose'.

In [14]:
df['purpose'].value_counts()

свадьба                                   797
на проведение свадьбы                     777
сыграть свадьбу                           774
операции с недвижимостью                  676
покупка коммерческой недвижимости         664
операции с жильем                         653
покупка жилья для сдачи                   653
операции с коммерческой недвижимостью     651
покупка жилья                             647
жилье                                     647
покупка жилья для семьи                   641
строительство собственной недвижимости    635
недвижимость                              634
операции со своей недвижимостью           630
строительство жилой недвижимости          626
покупка недвижимости                      624
строительство недвижимости                620
покупка своего жилья                      620
ремонт жилью                              612
покупка жилой недвижимости                607
на покупку своего автомобиля              505
заняться высшим образованием      

Приведем данные в столбце образования в нижний регистр.

In [15]:
df['education'] = df['education'].str.lower()

Убедимся, что данные преобразованы.

In [16]:
df['education'].value_counts()

среднее                15233
высшее                  5260
неоконченное высшее      744
начальное                282
ученая степень             6
Name: education, dtype: int64

Проверим наличие грубых дубликатов(повторов) методом duplicated().

In [17]:
df.duplicated().sum()

71

Удалим повторяющиеся стороки, а индексы перепишем в порядке возрастания, без пропусков.

In [18]:
df = df.drop_duplicates().reset_index(drop=True)

### Вывод

Для нахождения дубликатов в качестве отправной точки был выбран метод ручного поиска value_counts(). 

В столбце 'education' были обнаружены дубликаты с учетом регистра. Чтобы учесть такие дубликаты, все символы в строке приводятся к нижнему регистру вызовом метода lower().
В столбце 'purpose' были обнаружены дубликаты, записанные в разных формах.

### Лемматизация

Импортируем библиотеку с функцией лемматизации на русском языке — pymystem3, разработанная сотрудниками Яндекса для поиска слов, записанных в разных формах.

In [19]:
from pymystem3 import Mystem
m = Mystem()

Лемматизируем цели кредита.

In [20]:
df['lemmas'] = df['purpose'].apply(m.lemmatize)
df['lemmas'].value_counts()

[автомобиль, \n]                                          972
[свадьба, \n]                                             791
[на,  , проведение,  , свадьба, \n]                       768
[сыграть,  , свадьба, \n]                                 765
[операция,  , с,  , недвижимость, \n]                     675
[покупка,  , коммерческий,  , недвижимость, \n]           661
[операция,  , с,  , жилье, \n]                            652
[покупка,  , жилье,  , для,  , сдача, \n]                 651
[операция,  , с,  , коммерческий,  , недвижимость, \n]    650
[покупка,  , жилье, \n]                                   646
[жилье, \n]                                               646
[покупка,  , жилье,  , для,  , семья, \n]                 638
[строительство,  , собственный,  , недвижимость, \n]      635
[недвижимость, \n]                                        633
[операция,  , со,  , свой,  , недвижимость, \n]           627
[строительство,  , жилой,  , недвижимость, \n]            624
[покупка

### Вывод

Выделим основые цели получения кредита. 
Объединим в одну группу все значения где есть слова "жилье" и "недвижемость" и назвовем "Недвижижимость".
Если в цели кредита есть слово "образование", то обьединим этов группу "Образоавание".
Если в цели кредита есть слово "свадьба", то обьединим этов группу "Свадьба".
Если в цели кредита есть слово "автомобиль", то обьединим этов группу "Автомобиль".

### Категоризация данных

#### Выделим основные цели получения кредит.

In [21]:
def credit_purpose_status(credit_purpose):
    if 'образование' in credit_purpose:
            return 'Образование'
    if 'свадьба' in credit_purpose:
            return 'Свадьба'
    if 'автомобиль' in credit_purpose:
            return 'Автомобиль'
    if 'недвижимость' or 'жилье' in credit_purpose:
            return 'Недвижимость'
        
df['credit_purpose'] = df['lemmas'].apply(credit_purpose_status)
df['credit_purpose'].value_counts()

Недвижимость    10811
Автомобиль       4306
Образование      4013
Свадьба          2324
Name: credit_purpose, dtype: int64

#### Категоризируем клиентов банка по уровню дохода.

In [22]:
df['total_income'].describe().astype('int')

count      21454
mean      165319
std        98187
min        20667
25%       107623
50%       142594
75%       195820
max      2265604
Name: total_income, dtype: int64

In [23]:
def total_income_group(total_income):
    if total_income < 50000:
        return 'низкий'
    if total_income <= 100000:
        return 'средний'
    return 'высокий'

In [24]:
df['total_income_group'] = df['total_income'].apply(total_income_group)

### Вывод

Почти каждый второй кредит идет на недвижимость. Оснавная группа заемщиков имеет высокий среднемесечный доход(больше 100000 руб/мес)

### Шаг 3. Ответьте на вопросы

- Есть ли зависимость между наличием детей и возвратом кредита в срок?

In [25]:
df['children'].value_counts()

0    14091
1     4855
2     2128
3      330
4       41
5        9
Name: children, dtype: int64

Выведем среднее значение возврата кредита в срок и наличие детей.

In [26]:
df.groupby('children').agg({'purpose': 'count', 'debt': 'mean'}).sort_values('debt')

,purpose,debt
children,,
5,9,0.000000
0,14091,0.075438
3,330,0.081818
1,4855,0.091658
2,2128,0.094925
4,41,0.097561


### Вывод

Отсортирвав столбец с средним значением возврата кредита в срок, можно увидеть, что чаще всего возвращают кредит в срок заемщики без детей. Клиенты банка имеющие 5 детей абсолютно все возвращают кредит в срок, но их несоизмеримо мало по сравненю с другими, поэтому не стоит их учитывать.

- Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [27]:
df.groupby('family_status').agg({'purpose': 'count', 'debt': 'mean'}).sort_values('debt')

,purpose,debt
family_status,,
вдовец / вдова,959,0.065693
в разводе,1195,0.071130
женат / замужем,12339,0.075452
гражданский брак,4151,0.093471
Не женат / не замужем,2810,0.097509


### Вывод

Хуже всего возвращают кредит в срок люди, находящиеся в гражданском браке либо неженатые/незамужние.

- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [28]:
df.groupby('total_income_group').agg({'purpose': 'count', 'debt': 'mean'}).sort_values('debt')

,purpose,debt
total_income_group,,
низкий,372,0.061828
средний,4091,0.080909
высокий,16991,0.081631


### Вывод

Люди с доходом меньше 50000 руб/мес лучше всего возвращают кредит в срок.

- Как разные цели кредита влияют на его возврат в срок?

In [29]:
df.groupby('credit_purpose').agg({'purpose': 'count', 'debt': 'mean'}).sort_values('debt')

,purpose,debt
credit_purpose,,
Недвижимость,10811,0.072334
Свадьба,2324,0.080034
Образование,4013,0.092200
Автомобиль,4306,0.093590


### Вывод

Люди, которые берут кредит на автомобиль либо на образование, хуже всего возвращают кредит в срок.

### Шаг 4. Общий вывод

Клиенты банка, у которых нет детей чаще всего берут кредит, что логично, т.к они свои свободные деньги могут вложить в кредит, а не на детей и чаще всего в срок кредит возвращают.

Если посмотреть на зависимость семейного положения и возврата кредита в срок, то мы увидим, что чаще берут кредит, те кто состоит в официальном браке, но хуже всего кредит в срок возвращают незамужние/неженатые и находящиеся в гражданском браке.

Заемщики, имеющие доход ниже 50000 руб/мес лучше остальных возвращают кредит в срок.

Если люди взяли кредит на недвижимость, то они чаще возвращают его в срок, по сравнению кредитами на образование или автомобиль.